In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from torch import distributions
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [5]:
trials = 1000000

url = '../PRAlikeData/qutrit'+str(trials)+'RandomSic.npy'
#url2 = '/content/drive/MyDrive/data4Ent/PRA/gltest_'+str(trials)+'.npy'
#url3 = '/content/drive/MyDrive/data4Ent/PRA/qutritIdealBorns_tetra.npy'
#url = '/content/drive/MyDrive/data4Ent/MtoMdata/2qutritGlobDepo0.2betaTrial1^6KBorns_tetra.npy'

tomo = np.load(url, allow_pickle=True)
#gl = np.load(url2, allow_pickle= True)
#gl = np.reshape(gl, (1000,-1))

#ideals = np.load(url3)


In [6]:
tomo_train, tomo_temp = train_test_split(tomo, test_size=0.7, random_state=1)
tomo_valid,tomo_test = train_test_split(tomo_temp, test_size=0.93 ,random_state=1)

tomo_test, etc = train_test_split( tomo_test, test_size = 0.92 )

import random
valid = tomo_valid
train = tomo_train
test = tomo_test

random.shuffle(train)
random.shuffle(valid)
random.shuffle(test)


In [8]:

train.shape, valid.shape, test.shape


((6000, 18), (980, 18), (1041, 18))

In [9]:
class StatesDataset(Dataset):

    def __init__(self, x):
        super().__init__()
        self.x = x

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.x[index]


# Data Loading

In [10]:


train_dataset = StatesDataset(train)
evaluation_dataset = StatesDataset(valid)
test_dataset = StatesDataset(test)


Fix the seeds for reproducibility

In [11]:

torch.manual_seed(13)

# Model Definition

In [24]:
del(themodelbeta)

In [ ]:

class Netbeta(nn.Module):
    def __init__(self, state_local_d, num_low_triang, out_channel, nheads, dim_ff ):
        super().__init__()
      
        #ENCODING
        self.conv1 = nn.Conv1d(1, out_channel,2 ) #new input feature will be state_local_d**2 -1

        #DECODING
        self.enc_transf = nn.TransformerEncoderLayer(d_model = 8, nhead = nheads, dim_feedforward = dim_ff, batch_first = True) # hereh hidden_size must be D-out/9 to become d_out after flattening
        self.relu = torch.nn.ReLU()
        self.enc_stack = nn.TransformerEncoder(self.enc_transf, num_layers=1)
        self.T = torch.nn.Tanh()
      
        #diagonal part
        self.cholesky_diag = torch.nn.Linear(out_channel*8, state_local_d)

        # (n*n +n)/2 strictly lower diagonal elements
        self.cholesky_lower_t = torch.nn.Linear(out_channel*8, 2*num_low_triang)

    def forward(self, x):

        #ENCODING

        x = F.selu(self.conv1(x)) # out1 = (c1(inp))

        #DECODING

        x= self.g(self.enc_stack(x) )
        x = torch.flatten(x,1)

        diags = self.relu(self.cholesky_diag(x)) 
        off_diags = self.T(self.cholesky_lower_t(x))

        return diags, off_diags



# Training for beta version


In [26]:
local_dim = 3
dim = 1


In [28]:
num_killed_inputs = 0

input_dim = local_dim**(2*dim) - num_killed_inputs
batch_size = 1000

statedim = local_dim**dim
lowerval = (statedim**2 -statedim)/2
print(lowerval)

device = "cpu"
learning_rate = 0.0001
num_epochs = 10


torch.cuda.empty_cache()

netb = Netbeta(int(local_dim) , int(lowerval), 2,1,10) #it was 12 channels and 10 ff dim in all other tests
themodelbeta  = netb.to(device)
themodelbeta = themodelbeta.double()


def normalization_loss(input1,input2):

  single_vector = torch.cat((input1,input2),dim=1)
  batchsize = input1.shape[0]
  
  #return torch.abs(torch.sum(torch.norm(single_vector, p ='fro', dim=1))/batchsize -1 )
  return torch.mean(torch.norm(single_vector, p ='fro', dim=1))/batchsize 

3.0


In [29]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size,
    shuffle=True, 
    pin_memory=True)

valid_dataloader = torch.utils.data.DataLoader(
    evaluation_dataset, batch_size=batch_size,
    shuffle=True, 
    pin_memory=True)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size,
    shuffle=True, 
    pin_memory=True)

#gl_dataloader = torch.utils.data.DataLoader(
#    gl_dataset, batch_size = batch_size, shuffle =False,
#    pin_memory = True)


In [30]:

rec_loss = torch.nn.MSELoss()
optimizer = optim.RMSprop(themodelbeta.parameters(), lr=learning_rate)

val_hist = []
train_hist = []

for epoch in range(num_epochs):
    train_loss = 0
    for inputs in train_dataloader:
       
        themodelbeta.train()

        inputs = inputs.to(device)
        optimizer.zero_grad()
       
        low_t = inputs[:, local_dim**(2*dim) + local_dim**dim - num_killed_inputs :]
        diagel = inputs[:,local_dim**(2*dim): local_dim**(2*dim) + local_dim**(dim) - num_killed_inputs]

        #INPUTS RESHAPING

        newin = torch.reshape(inputs[:,:local_dim**(2*dim) - num_killed_inputs ], (inputs.shape[0],1,local_dim**(2*dim )) )
        out1, out2 = themodelbeta(newin)
        
        print(out1.shape, out2.shape, diagel.shape,low_t.shape)
        
        #out1 = themodelbeta(newin)
        loss = rec_loss(out1, diagel) + rec_loss(out2, low_t) + normalization_loss(out1,out2)
        #loss = rec_loss(out1,inputs[:,9:]) 

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    valid_loss = 0
    for inputs in valid_dataloader:
        with torch.no_grad():
            themodelbeta.eval()
            optimizer.zero_grad()

            inputs = inputs.to(device)
            
            low_t = inputs[:, local_dim**(2*dim) + local_dim**dim - num_killed_inputs :]
            diagel = inputs[:,local_dim**(2*dim): local_dim**(2*dim) + local_dim**(dim) - num_killed_inputs]

           
            newinval = torch.reshape(inputs[:,:local_dim**(2*dim) - num_killed_inputs ], (inputs.shape[0],1,local_dim**(2*dim)) )
            out1, out2 = themodelbeta(newinval)

            loss = rec_loss(out1, diagel) + rec_loss(out2, low_t) + normalization_loss(out1,out2)
            #loss = rec_loss(out2,inputs[:,9:]) 

            valid_loss += loss.item()

    
    print(epoch, train_loss/len(train_dataloader), valid_loss/len(valid_dataloader))
    val_hist.append(valid_loss/len(valid_dataloader))
    train_hist.append(train_loss/len(train_dataloader))

torch.Size([1000, 3]) torch.Size([1000, 6]) torch.Size([1000, 3]) torch.Size([1000, 6])
torch.Size([1000, 3]) torch.Size([1000, 6]) torch.Size([1000, 3]) torch.Size([1000, 6])
torch.Size([1000, 3]) torch.Size([1000, 6]) torch.Size([1000, 3]) torch.Size([1000, 6])
torch.Size([1000, 3]) torch.Size([1000, 6]) torch.Size([1000, 3]) torch.Size([1000, 6])
torch.Size([1000, 3]) torch.Size([1000, 6]) torch.Size([1000, 3]) torch.Size([1000, 6])
torch.Size([1000, 3]) torch.Size([1000, 6]) torch.Size([1000, 3]) torch.Size([1000, 6])
0 0.44120999508130904 0.3886148075177755
torch.Size([1000, 3]) torch.Size([1000, 6]) torch.Size([1000, 3]) torch.Size([1000, 6])
torch.Size([1000, 3]) torch.Size([1000, 6]) torch.Size([1000, 3]) torch.Size([1000, 6])
torch.Size([1000, 3]) torch.Size([1000, 6]) torch.Size([1000, 3]) torch.Size([1000, 6])
torch.Size([1000, 3]) torch.Size([1000, 6]) torch.Size([1000, 3]) torch.Size([1000, 6])
torch.Size([1000, 3]) torch.Size([1000, 6]) torch.Size([1000, 3]) torch.Size([1

# Test. Quantum fidelity state reconstruction

In [ ]:
def return_matrix_elements(stuff,d,local_dim): 

	diag_len = local_dim**d
	#print(diag_len)

	non_diag_len = (local_dim**(2*d) -diag_len)/2
	#print(non_diag_len)

	#head-up: local dim**(2d) is also the length of the initial state. The total dimension of the input is 2Xlocal dim**(2d)

	diagel = stuff[local_dim**(2*d)  - num_killed_inputs : local_dim**(2*d)  + diag_len  - num_killed_inputs]
	offd = stuff[ local_dim**(2*d)  + diag_len -num_killed_inputs:  ]
	#print(len(imag))
	return diagel, offd



In [ ]:
def rebuild_last(diags,offd,ind):
    '''''
    input.
    diags : diags elements
    offds : offdiagonal elements
    rhoshape : just the dm dimension, e.g. number of rows
    localDim : the number of level of each particle
    numParticles : the total number of particles for tensor product

    output.
    m : reconstructed cholesky decoposition matrix
    '''''
    d = len(diags)
    eye = np.eye(d,d)
    mat = np.zeros((d,d),dtype = complex)

    offvalues = [ (a+1j*b) for (a,b) in zip(offd[:int(len(offd)/2 ) ], offd[int(len(offd)/2):])  ]
    mat[ind[0], ind[1]] = offvalues

    return mat + eye*diags

In [ ]:
#just generate ones the index for the recosntruction using a dull matrix

import qutip as qt
def fid(a,b):
  fid = qt.fidelity(qt.Qobj(a), qt.Qobj(b))
  return torch.tensor(fid)

#parameters
buresdistance =0
qfid =0
j=0
hs = []
fids =[]

# generate support diagonal matrix for reconstruction
e = np.eye(local_dim**(dim),local_dim**(dim)) 
print(e.shape)

#indeces for the reconstruction function
ind = np.tril_indices_from(e,k=-1)



(3, 3)


# new test phase

In [ ]:


with torch.no_grad():
  for inputs in test_dataloader:
  
    inputs = inputs.to(device)

    #out_d is the diagonal, out_tr the triangluar values, real and complex altogether
    newin = torch.reshape(inputs[:,:local_dim**(2*dim) - num_killed_inputs ], (inputs.shape[0],1,local_dim**2) )

    out_d, out_tr = themodelbeta(newin) 

    for i in range(out_d.shape[0]):

      #reconstructing cholesky from dataset input array. NO MORE NEEDED
      diagel, offd = return_matrix_elements( inputs[i].cpu().numpy(), dim, local_dim) 

      original_chol = rebuild_last(diagel, offd , ind)

      #reconstructing cholesky from neural network outputs

      nn_diag = out_d[i].cpu().numpy()      
      nn_offd = out_tr[i,:int(out_tr.shape[1])].cpu().numpy()
      
      nn_chol = rebuild_last(nn_diag, nn_offd, ind)

      cholo=nn_chol@ nn_chol.conj().T
      #purity = np.trace(cholo@cholo)
      norm = np.trace(cholo)
      #norm = 1

      #fidelity between reconstructed and originals

      fids.append(fid(original_chol@ original_chol.conj().T,cholo/norm) )
      hs.append(qt.hilbert_dist(qt.Qobj(cholo/norm), qt.Qobj(original_chol@ original_chol.conj().T)))

print(f"averaged quantum fidelity {np.mean(fids): .4f} ")#dont forget the j to divide by

print(np.std(fids))
print(f"hilbert schmidt distance average {np.mean(hs): .4f}")
print(f"hilbert schmidt distance std {np.std(hs): .4f}")

averaged quantum fidelity  0.9990 
0.0012061989
hilbert schmidt distance average  0.0020
hilbert schmidt distance std  0.0029


In [ ]:
del(themodelbeta)

GUILLEM FILES

In [ ]:
test_gl = gl

gl_dataset = StatesDataset(test_gl)
gl_dataloader = torch.utils.data.DataLoader(
    gl_dataset, batch_size = batch_size, shuffle =False,
    pin_memory = True)

In [ ]:
glFile = []

name = 'qutritTestSet_6Ktrain_'+str(trials)+'_CNNtransformer.npy'

with torch.no_grad():
  for inputs in gl_dataloader:
  
    inputs = inputs.to(device)

    #out_d is the diagonal, out_tr the triangluar values, real and complex altogether
    newin = torch.reshape(inputs[:,: ], (inputs.shape[0],1,local_dim**2) )

    out_d, out_tr = themodelbeta(newin) 

    for i in range(out_d.shape[0]):

      #reconstructing cholesky from neural network outputs

      nn_diag = out_d[i].cpu().numpy()      
      nn_offd = out_tr[i,:int(out_tr.shape[1])].cpu().numpy()
      
      nn_chol = rebuild_last(nn_diag, nn_offd, ind)

      cholo=nn_chol@ nn_chol.conj().T
      #purity = np.trace(cholo@cholo)
      norm = np.trace(cholo)

      glFile.append(cholo/norm)
  
  np.save(name, glFile)

SMALL TESTS AREA

let's do embeddings

In [ ]:
latent_separable_representations = []

for inputs in separable_test_dataloader:

    with torch.no_grad():
        auto_encoder.eval()

        inputs = inputs.to(device)
        optimizer.zero_grad()
        encoded_x = auto_encoder.encoder(inputs)
        latent_separable_representations.extend(encoded_x.cpu().detach().numpy())

latent_entangled_representations = []

for inputs in entangled_test_dataloader:

    with torch.no_grad():
        auto_encoder.eval()

        inputs = inputs.to(device)
        optimizer.zero_grad()
        encoded_x = auto_encoder.encoder(inputs)
        latent_entangled_representations.extend(encoded_x.cpu().detach().numpy())

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
silhouette_score(X, y)